In [1]:
import os

dataset_path = r"C:\Users\ASUS\OneDrive\Documents\Study\Project ML 1\pest\train"

if not os.path.exists(dataset_path):
    print("❌ ERROR: Dataset path does not exist.")
else:
    print("✅ Dataset found!")
    print("🔍 Checking folder contents...\n")


    for category in sorted(os.listdir(dataset_path)):
        category_path = os.path.join(dataset_path, category)

        if not os.path.isdir(category_path):
             print(f"⚠️ Warning: {category} is not a folder!")
        else:
            images = os.listdir(category_path)
            print(f"📂 {category} - {len(images)} images")

✅ Dataset found!
🔍 Checking folder contents...

📂 aphids - 300 images
📂 armyworm - 300 images
📂 beetle - 300 images
📂 bollworm - 300 images
📂 grasshopper - 300 images
📂 mites - 300 images
📂 mosquito - 300 images
📂 sawfly - 300 images
📂 stem_borer - 300 images


In [2]:
import os

dataset_path = r"C:\Users\ASUS\OneDrive\Documents\Study\Project ML 1\pest\train"

for category in sorted(os.listdir(dataset_path)):
    category_path = os.path.join(dataset_path, category)

    if os.path.isdir(category_path) and len(os.listdir(category_path)) == 0:
        print(f"🗑 Removing empty folder: {category_path}")
        os.rmdir(category_path)

print("✅ Empty folders removed!")

✅ Empty folders removed!


In [3]:
!pip install torch torchvision
import torch
import torch.nn as nn
import torchvision.transforms as transforms
import torchvision.datasets as datasets
import torch.optim as optim
from torchvision import models


dataset_path = r"C:\Users\ASUS\OneDrive\Documents\Study\Project ML 1\pest\train"


transform = transforms.Compose([
    transforms.Resize((128, 128)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])


train_dataset = datasets.ImageFolder(root=dataset_path, transform=transform)
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=32, shuffle=True)


class_labels = train_dataset.classes
print("Detected Classes:", class_labels)


device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = models.mobilenet

Detected Classes: ['aphids', 'armyworm', 'beetle', 'bollworm', 'grasshopper', 'mites', 'mosquito', 'sawfly', 'stem_borer']


In [4]:
!pip install torch torchvision
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision.transforms as transforms
import torchvision.datasets as datasets
from torchvision import models
import os


dataset_path = r"C:\Users\ASUS\OneDrive\Documents\Study\Project ML 1\pest\train"

model_save_path = r"C:\Users\ASUS\OneDrive\Documents\Study\Project ML 1\Model_Save_Path"
os.makedirs(model_save_path, exist_ok=True)

model_save_file = os.path.join(model_save_path, "pest_model.pth")


class_labels = sorted(os.listdir(dataset_path))
print(f"Detected Classes: {class_labels}")


transform = transforms.Compose([
    transforms.Resize((128, 128)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])


train_dataset = datasets.ImageFolder(root=dataset_path, transform=transform)
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=32, shuffle=True)


device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = models.mobilenet_v2(weights=None)
model.classifier[1] = nn.Linear(model.classifier[1].in_features, len(class_labels))
model.to(device)


criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)


num_epochs = 10  
print("🚀 Training Started...")
model.train()
for epoch in range(num_epochs):
    running_loss = 0.0
    for images, labels in train_loader:
        images, labels = images.to(device), labels.to(device)
        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()

    print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {running_loss/len(train_loader):.4f}")



torch.save(model.state_dict(), model_save_file)
print(f"🎉 Training Complete! Model saved to: {model_save_path}")

Detected Classes: ['aphids', 'armyworm', 'beetle', 'bollworm', 'grasshopper', 'mites', 'mosquito', 'sawfly', 'stem_borer']
🚀 Training Started...
Epoch [1/10], Loss: 1.9749
Epoch [2/10], Loss: 1.3785
Epoch [3/10], Loss: 0.8167
Epoch [4/10], Loss: 0.5248
Epoch [5/10], Loss: 0.3952
Epoch [6/10], Loss: 0.3040
Epoch [7/10], Loss: 0.1841
Epoch [8/10], Loss: 0.1052
Epoch [9/10], Loss: 0.1786
Epoch [10/10], Loss: 0.1651
🎉 Training Complete! Model saved to: C:\Users\ASUS\OneDrive\Documents\Study\Project ML 1\Model_Save_Path


In [5]:
import os
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision.transforms as transforms
import torchvision.datasets as datasets
from torchvision import models
from PIL import Image
import numpy as np
import cv2
import gradio as gr
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import classification_report, confusion_matrix

# 🔧 Paths
dataset_path = r"C:\Users\ASUS\OneDrive\Documents\Study\Project ML 1\pest\train"
model_path = r"C:\Users\ASUS\OneDrive\Documents\Study\Project ML 1\Model_Save_Path\pest_model.pth"
class_labels = sorted(os.listdir(dataset_path))

# 🔄 Image Transform
transform = transforms.Compose([
    transforms.Resize((128, 128)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

# 🧠 Load Model
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = models.mobilenet_v2(weights=None)
model.classifier[1] = nn.Linear(model.classifier[1].in_features, len(class_labels))
model.load_state_dict(torch.load(model_path, map_location=device))
model.to(device)
model.eval()

# 🐛 Pest Info
pest_info = {
    "aphids": {"treatment": "Use neem oil spray.", "prevention": "Introduce ladybugs.", "medicine": "Neem Oil, Pyrethrin Spray"},
    "armyworm": {"treatment": "Apply Bacillus thuringiensis.", "prevention": "Crop rotation.", "medicine": "BT-Kurstaki, Spinosad"},
    "beetle": {"treatment": "Use insecticidal soap.", "prevention": "Keep field clean.", "medicine": "Carbaryl, Pyrethroid"},
    "bollworm": {"treatment": "Apply spinosad-based insecticide.", "prevention": "Use pheromone traps.", "medicine": "Spinosad, Emamectin Benzoate"},
    "grasshopper": {"treatment": "Spray garlic-based repellent.", "prevention": "Encourage natural predators.", "medicine": "Nosema locustae bait, Pyrethroids"},
    "mites": {"treatment": "Use sulfur-based miticide.", "prevention": "Avoid dust buildup.", "medicine": "Avermectin, Sulfur Spray"},
    "mosquito": {"treatment": "Apply larvicides in stagnant water.", "prevention": "Remove standing water.", "medicine": "Bti (Bacillus thuringiensis israelensis)"},
    "sawfly": {"treatment": "Use pyrethrin spray.", "prevention": "Introduce parasitic wasps.", "medicine": "Pyrethrin, Spinosad"},
    "stem_borer": {"treatment": "Apply carbaryl dust.", "prevention": "Resistant crop varieties.", "medicine": "Carbaryl, Chlorpyrifos"}
}

# 🔍 Optional: OpenCV color detection
def detect_pest_opencv(image):
    img = np.array(image)
    img = cv2.cvtColor(img, cv2.COLOR_RGB2BGR)
    hsv = cv2.cvtColor(img, cv2.COLOR_BGR2HSV)

    lower_brown = np.array([10, 50, 20])
    upper_brown = np.array([30, 255, 200])
    lower_black = np.array([0, 0, 0])
    upper_black = np.array([180, 255, 50])

    mask_brown = cv2.inRange(hsv, lower_brown, upper_brown)
    mask_black = cv2.inRange(hsv, lower_black, upper_black)
    combined_mask = mask_brown | mask_black

    contours, _ = cv2.findContours(combined_mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    return any(cv2.contourArea(contour) > 500 for contour in contours)

# 🧠 Prediction
def predict_pest(image):
    if not detect_pest_opencv(image):
        return "✅ No Pest Detected! Your crop is healthy. 🌿"

    image = image.convert("RGB")
    image = transform(image).unsqueeze(0).to(device)
    with torch.no_grad():
        output = model(image)
        probabilities = torch.nn.functional.softmax(output[0], dim=0)
        confidence, pred_idx = torch.max(probabilities, dim=0)
        predicted_pest = class_labels[pred_idx.item()]

    treatment = pest_info.get(predicted_pest, {}).get("treatment", "No data")
    prevention = pest_info.get(predicted_pest, {}).get("prevention", "No data")
    medicine = pest_info.get(predicted_pest, {}).get("medicine", "No data")

    return f"""🐛 Pest Detected: {predicted_pest} ({confidence.item()*100:.2f}% confidence)

🛠 Treatment: {treatment}
🌱 Prevention: {prevention}
💊 Recommended Medicine: {medicine}
"""

# 📊 Show Model Metrics (Formatted)
def show_model_metrics():
    test_dataset = datasets.ImageFolder(root=dataset_path, transform=transform)
    test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=32, shuffle=False)

    all_preds, all_labels = [], []

    model.eval()
    with torch.no_grad():
        for images, labels in test_loader:
            images = images.to(device)
            outputs = model(images)
            _, predicted = torch.max(outputs.data, 1)
            all_preds.extend(predicted.cpu().numpy())
            all_labels.extend(labels.cpu().numpy())

    # 📋 Classification Report
    report_str = classification_report(all_labels, all_preds, target_names=class_labels, digits=2)
    formatted_report = f"```\n{'Class':>15}  {'Precision':>10}  {'Recall':>7}  {'F1-Score':>9}  {'Support':>8}\n"
    formatted_report += "-" * 60 + "\n"
    for line in report_str.split("\n")[2:-5]:
        if line.strip():
            formatted_report += f"{line.strip()}\n"
    formatted_report += "-" * 60 + "\n"
    for line in report_str.split("\n")[-5:]:
        if line.strip():
            formatted_report += f"{line.strip()}\n"
    formatted_report = f"```\n{formatted_report}\n```"

    # 📉 Confusion Matrix
    cm = confusion_matrix(all_labels, all_preds)
    fig1, ax1 = plt.subplots(figsize=(8, 6))
    sns.heatmap(cm, annot=True, fmt="d", cmap="Blues", xticklabels=class_labels, yticklabels=class_labels, ax=ax1)
    ax1.set_title("Confusion Matrix")
    ax1.set_xlabel("Predicted")
    ax1.set_ylabel("True")

    # 📊 Metrics Bar Chart
    report_dict = classification_report(all_labels, all_preds, target_names=class_labels, output_dict=True)
    precision = [report_dict[cls]["precision"] for cls in class_labels]
    recall = [report_dict[cls]["recall"] for cls in class_labels]
    f1 = [report_dict[cls]["f1-score"] for cls in class_labels]

    fig2, ax2 = plt.subplots(figsize=(10, 6))
    x = range(len(class_labels))
    ax2.bar(x, precision, width=0.25, label='Precision')
    ax2.bar([i + 0.25 for i in x], recall, width=0.25, label='Recall')
    ax2.bar([i + 0.50 for i in x], f1, width=0.25, label='F1 Score')
    ax2.set_xticks([i + 0.25 for i in x])
    ax2.set_xticklabels(class_labels, rotation=45)
    ax2.set_title("Per-Class Precision, Recall & F1")
    ax2.legend()

    return formatted_report, fig1, fig2

# 🎛️ Gradio App
with gr.Blocks() as demo:
    gr.Markdown("## 🌿 Advanced Pest Detection using AI & OpenCV")

    with gr.Tab("🔍 Pest Detection"):
        image_input = gr.Image(type="pil", label="Upload Crop Image")
        output_text = gr.Textbox(label="Detection Result")
        detect_btn = gr.Button("🔍 Detect Pest")
        detect_btn.click(fn=predict_pest, inputs=image_input, outputs=output_text)

    with gr.Tab("📊 Model Metrics"):
        metrics_button = gr.Button("📊 Show Model Metrics")
        metrics_output_text = gr.Textbox(label="Classification Report")
        metrics_plot1 = gr.Plot(label="Confusion Matrix")
        metrics_plot2 = gr.Plot(label="Per-Class Metrics")
        metrics_button.click(fn=show_model_metrics, inputs=[], outputs=[metrics_output_text, metrics_plot1, metrics_plot2])

demo.launch(share=True)



* Running on local URL:  http://127.0.0.1:7860
* Running on public URL: https://73c515551dfc403d3b.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
